In [1]:
import pandas as pd
import numpy as np
import tpqoa

In [2]:
api = tpqoa.tpqoa("oanda.cfg")

In [3]:
api.get_history(instrument="EUR_USD", start="2020-09-21", end="2020-09-24", granularity="H1", price="B")

,volume,complete,o,h,l,c
time,,,,,,
2020-09-21 00:00:00+00:00,1632,True,1.18447,1.18602,1.18423,1.18587
2020-09-21 01:00:00+00:00,1908,True,1.18588,1.18611,1.18536,1.18554
2020-09-21 02:00:00+00:00,1799,True,1.18555,1.18619,1.18554,1.18580
2020-09-21 03:00:00+00:00,1380,True,1.18581,1.18668,1.18559,1.18668
2020-09-21 04:00:00+00:00,965,True,1.18668,1.18712,1.18652,1.18660
...,...,...,...,...,...,...
2020-09-23 19:00:00+00:00,5172,True,1.16568,1.16636,1.16508,1.16620
2020-09-23 20:00:00+00:00,1775,True,1.16617,1.16629,1.16541,1.16592
2020-09-23 21:00:00+00:00,110,True,1.16592,1.16615,1.16551,1.16589


In [4]:
api.stream_data("EUR_USD", stop=10)

2020-12-04T14:51:25.346706701Z 1.2162 1.21628
2020-12-04T14:51:25.545571622Z 1.21622 1.21632
2020-12-04T14:51:26.032995787Z 1.21624 1.2163
2020-12-04T14:51:27.199976096Z 1.21625 1.21632
2020-12-04T14:51:29.394475931Z 1.21626 1.21633
2020-12-04T14:51:29.416509583Z 1.21628 1.21635
2020-12-04T14:51:30.228273563Z 1.2163 1.21636
2020-12-04T14:51:34.772408872Z 1.21631 1.21638
2020-12-04T14:51:36.264553437Z 1.2163 1.21636
2020-12-04T14:51:37.153756497Z 1.21633 1.2164


In [5]:
class CloneClass(tpqoa.tpqoa):
    def on_success(self, time, bid, ask):
        print("Time: {} | Bid: {} | Ask: {}".format(time, bid, ask))

In [6]:
api = CloneClass("oanda.cfg")

In [7]:
api.get_instruments()[:5]

[('Silver/EUR', 'XAG_EUR'),
 ('CAD/SGD', 'CAD_SGD'),
 ('Silver', 'XAG_USD'),
 ('UK 100', 'UK100_GBP'),
 ('GBP/NZD', 'GBP_NZD')]

In [8]:
api.stream_data("EUR_USD", stop=5)

Time: 2020-12-04T14:51:37.392411181Z | Bid: 1.21635 | Ask: 1.21641
Time: 2020-12-04T14:51:38.712222148Z | Bid: 1.21634 | Ask: 1.2164
Time: 2020-12-04T14:51:39.079302158Z | Bid: 1.21635 | Ask: 1.21641
Time: 2020-12-04T14:51:39.620414385Z | Bid: 1.21636 | Ask: 1.21642
Time: 2020-12-04T14:51:39.696288084Z | Bid: 1.21635 | Ask: 1.21641


In [9]:
class GetTickData(tpqoa.tpqoa):

    def __init__(self, confiig_file, bar_lenght):
        super().__init__(confiig_file)
        self.tick_data = pd.DataFrame()
        self.bar_lenght = bar_lenght

    def on_success(self, time, bid, ask):
        print(time, bid, ask)
        df = pd.DataFrame({"bid": bid, "ask": ask, "mid": (ask+bid)/2}, index=[pd.to_datetime(time)])
        self.tick_data = self.tick_data.append(df)
        self.data = self.tick_data.resample(self.bar_lenght, label="right").last().ffill().iloc[:-1]

In [10]:
td = GetTickData("oanda.cfg", "5s")

In [11]:
td.stream_data("EUR_USD", stop=10)

2020-12-04T14:51:40.658603287Z 1.21637 1.21643
2020-12-04T14:51:41.525888290Z 1.21633 1.2164
2020-12-04T14:51:42.004265740Z 1.21633 1.21641
2020-12-04T14:51:42.416200270Z 1.21636 1.21642
2020-12-04T14:51:43.248613542Z 1.21636 1.21644
2020-12-04T14:51:43.305240093Z 1.21635 1.21642
2020-12-04T14:51:44.232757851Z 1.21634 1.2164
2020-12-04T14:51:44.511216680Z 1.21629 1.21635
2020-12-04T14:51:44.659838159Z 1.21626 1.21632
2020-12-04T14:51:47.309231426Z 1.21627 1.21633


In [12]:
td.data

,bid,ask,mid
2020-12-04 14:51:45+00:00,1.21626,1.21632,1.21629


In [14]:
td.create_order("EUR_USD", units=100000)



 {'id': '48', 'time': '2020-12-04T14:52:23.802694545Z', 'userID': 17115891, 'accountID': '101-011-17115891-001', 'batchID': '47', 'requestID': '42769435898288915', 'type': 'ORDER_FILL', 'orderID': '47', 'instrument': 'EUR_USD', 'units': '100000.0', 'gainQuoteHomeConversionFactor': '1.0', 'lossQuoteHomeConversionFactor': '1.0', 'price': 1.21627, 'fullVWAP': 1.21627, 'fullPrice': {'type': 'PRICE', 'bids': [{'price': 1.21621, 'liquidity': '1000000'}, {'price': 1.2162, 'liquidity': '2000000'}, {'price': 1.21619, 'liquidity': '2000000'}, {'price': 1.21617, 'liquidity': '5000000'}], 'asks': [{'price': 1.21627, 'liquidity': '1000000'}, {'price': 1.21629, 'liquidity': '2000000'}, {'price': 1.2163, 'liquidity': '2000000'}, {'price': 1.21631, 'liquidity': '5000000'}], 'closeoutBid': 1.21617, 'closeoutAsk': 1.21631}, 'reason': 'MARKET_ORDER', 'pl': '0.0', 'financing': '0.0', 'commission': '0.0', 'guaranteedExecutionFee': '0.0', 'accountBalance': '100123.1633', 'tradeOpened': {'tradeID': '48', '